```
Author: Michael Gao
Last Updated: 06/04/2019
```
#### TODO: 
This document is a work in progress. Planned expansions include: 

    1. Timestamps/Timezones, etc.
    2. Multi-indices
    3. Keeping data when using .aggregate

# Importing 

When importing pandas, please use the following convention

In [2]:
import pandas as pd

# Pandas

Pandas is the canonical library that is used to work with tabular data in python. Pandas is powerful and has many convenient functions for working with dataframes, and is a staple of data science in python. Because it is written to be flexible and handle a variety of use cases, there are many ways to accomplish the same task in Pandas. While this may seem like a good thing, it can be difficult when working in a team environment. In addition, there are ways of getting from point A to point B that are not always ideal. The purpose of this document is to provide you with a set of guidelines to follow when using the Pandas library.

Although some of these choices are to protect you from unintended behavior (for example incorrect subsetting), others are to make sure that any code you submit will adhere to a standard style that the Duke Institute for Health Innovation has chosen to adhere to. A core tenant of almost all of these guidelines is simply to err on the side of being *more* explicit about what you are trying to do rather than being concise.

## Accessing Columns

When accessing columns in Pandas, use the bracket operator `[]` rather than using dot notation `.`. This is especially important when column names cannot be coerced into the dot notation. Let's look at an example: 

In [81]:
badly_named_df = pd.DataFrame({'column 1': [1,2,3],
                               'column_2': [2,3,4]})

In [82]:
badly_named_df

,column 1,column_2
0,1,2
1,2,3
2,3,4


Here, you can call

In [85]:
badly_named_df.column_2

0    2
1    3
2    4
Name: column_2, dtype: int64

but you can't use the same notation to call column 1

In [86]:
badly_named_df.column 1

SyntaxError: invalid syntax (<ipython-input-86-cc55a3ae23c4>, line 1)

In [89]:
[i for i in dir(badly_named_df) if 'column' in i] # not even an option

['_combine_match_columns',
 '_reindex_columns',
 '_sanitize_column',
 'column_2',
 'columns']

However, using the bracket notation, we never face this issue 

In [90]:
badly_named_df['column 1']

0    1
1    2
2    3
Name: column 1, dtype: int64

In [91]:
badly_named_df['column_2']

0    2
1    3
2    4
Name: column_2, dtype: int64

In general, we'd like to avoid having column names with spaces, but sometimes this is inevitable due to the structure of data that is being read in. Therefore, using bracket notation always will prevent any inconsistencies in your code.

## Performing `inplace` operations

There are many functions available in Pandas that allow you to perform inplace operations, meaning that the object that you are performing a function on will actually be changed even without explicit assignment. 
> **Avoid inplace = True.** 

A common example is removing rows that contains NA values:

In [106]:
missing_df = pd.DataFrame({'column_1': [1,2,3,4,5],
                           'column_2': [2,3,4,5, None]})
missing_df

,column_1,column_2
0,1,2.0
1,2,3.0
2,3,4.0
3,4,5.0
4,5,NaN


In [110]:
# Correct: 
missing_df = missing_df.dropna(axis = 0)

In [104]:
missing_df

,column_1,column_2
0,1,2.0
1,2,3.0
2,3,4.0
3,4,5.0


In [108]:
missing_df = pd.DataFrame({'column_1': [1,2,3,4,5],
                           'column_2': [2,3,4,5, None]})

# incorrect -- AVOID:
missing_df.dropna(axis = 0, inplace = True)

In [109]:
missing_df

,column_1,column_2
0,1,2.0
1,2,3.0
2,3,4.0
3,4,5.0


## Subsetting

## 1. `.loc`

One of the most common operations in Pandas is subsetting a `DataFrame` object. However, there are many different ways that this operation is permitted. 
> **In general, use the `.loc` method to subset DataFrames**, especially when you plan to make modifications

There are many errors that can be prevented if you follow this general rule. Here is an example:

#### Create a DataFrame

In [49]:
example_df = pd.DataFrame({'patient_id': [123456, 234567, 345678, 456789],
                           'age': [20, 30, 40, 50],
                           'gender': ['M', 'F', 'M', 'F']})

example_df

,patient_id,age,gender
0,123456,20,M
1,234567,30,F
2,345678,40,M
3,456789,50,F


Now imagine we would like to subset this for all patients who are older than 35 and we want to return only the ID and the gender of the patient. The correct way to handle this is:

In [50]:
# Correct
correct_df = example_df.loc[example_df['age'] > 35, ['patient_id', 'gender']]
correct_df

,patient_id,gender
2,345678,M
3,456789,F


In [65]:
# Also correct:
correct_df = example_df.loc[example_df['age'] > 35, :]
correct_df.loc[:, ['patient_id', 'gender']]

,patient_id,gender
2,345678,M
3,456789,F


### A note on `:`

The `.loc` operator lets you specify which rows you'd like to select followed by which columns. Leaving the columns blank would also have worked in the above example. i.e.

In [68]:
example_df.loc[example_df['age'] > 35]

,patient_id,age,gender
2,345678,40,M
3,456789,50,F


However, because we want to err on the side of being explicit, please use the `:` to indicate that all columns should be returned 

The reverse direction will not even work if you do not specify which columns you would like. For example,

In [73]:
# Not valid
example_df.loc[, 'age']

SyntaxError: invalid syntax (<ipython-input-73-38e58c764b4b>, line 2)

In [75]:
# Valid
example_df.loc[:,['age']]


,age
0,20
1,30
2,40
3,50


please make sure you return the correct type of data structure that you'd like. Note that returning a single column without the extra brackets `[]` will result in a `pd.Series` being returned, while including the brackets will return a `pd.DataFrame`

In [76]:
type(example_df.loc[:, ['age']])

pandas.core.frame.DataFrame

In [77]:
type(example_df.loc[:, 'age'])

pandas.core.series.Series

### What if I don't use `.loc`?

In [52]:
# Incorrect
incorrect_df = example_df[example_df['age'] > 35]
incorrect_df[['patient_id', 'gender']]

,patient_id,gender
2,345678,M
3,456789,F


Now you might say, Michael, aren't you being annoying? Those do the same thing! Maybe, but here is an example of where the last case might not be ideal. Let's say you wanted to make sure that all patients older than 35 were actually Male (there may have been a mistake in extracting the data).

In [53]:
example_df.loc[example_df['age'] > 35, 'gender'] = 'M'

In [54]:
example_df

,patient_id,age,gender
0,123456,20,M
1,234567,30,F
2,345678,40,M
3,456789,50,M


No problem! What about the last method?

In [57]:
# Reset the dataframe
example_df = pd.DataFrame({'patient_id': [123456, 234567, 345678, 456789],
                           'age': [20, 30, 40, 50],
                           'gender': ['M', 'F', 'M', 'F']})

# no .loc
example_df[example_df['age'] > 35]['gender'] = 'M'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [58]:
example_df

,patient_id,age,gender
0,123456,20,M
1,234567,30,F
2,345678,40,M
3,456789,50,F


Hm, seems like this didn't actually change what we wanted! To fully understand why this behavior occurs, feel free to read the following article:

[SettingWithCopy Warning](https://www.dataquest.io/blog/settingwithcopywarning/)

In either case, use the `.loc` operator and in many cases you will avoid any of the above issues. In our production code, we will often set the following option: `pd.set_option('mode.chained_assignment', 'raise')` which will make it so that scripts will error out if they encounter this warning.

In [61]:
with pd.option_context('mode.chained_assignment', 'raise'):
    example_df[example_df['age'] > 35]['gender'] = 'M'

SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

## 2. Multiple conditions 

When subsetting on multiple conditions, it can sometimes be hard to parse exactly what is going on due to the large volume of text that it often takes to perform these multiple conditions. 
 > **Use multiple lines of code to show how you are subsetting a DataFrame multiple times**, especially with more than 2 conditions


In [62]:
example_df

,patient_id,age,gender
0,123456,20,M
1,234567,30,F
2,345678,40,M
3,456789,50,F


Let's say we want to subset patients who are female and over the age of 30. We can accomplish this in two separate ways: 

In [64]:
example_df.loc[(example_df['age'] > 30) & (example_df['gender'] == 'F'), :]

,patient_id,age,gender
3,456789,50,F


This doesn't look too bad, but as you might imagine it can get tricky as we add more conditions. For example, let's say we want 
``` 
 1. age > 30
 2. gender == "F"
 3. patient_id > 200000
 4. return age and gender columns
```

In [78]:
example_df.loc[(example_df['age'] > 30) & (example_df['gender'] == 'F') & (example_df['patient_id'] > 200000), ['age', 'gender']]

,age,gender
3,50,F


We can make this easier to read by breaking it into multiple lines of code

In [79]:
example_df.loc[(example_df['age'] > 30) 
               & (example_df['gender'] == 'F') 
               & (example_df['patient_id'] > 200000)
               , ['age', 'gender']]

,age,gender
3,50,F


In general, we will follow [PEP-8](https://www.python.org/dev/peps/pep-0008/#id20) guidelines for whether the operators come before or after the statements (they should come before)

It is also acceptable to simply break up the subsetting operations into multiple steps. i.e. 

In [80]:
subset_df = example_df.loc[example_df['age'] > 30, :]
subset_df = subset_df.loc[subset_df['gender'] == 'F', :]
subset_df = subset_df.loc[subset_df['patient_id'] > 200000, :]
subset_df = subset_df.loc[:, ['age', 'gender']]

subset_df

,age,gender
3,50,F


## Joins

There are many ways to join `DataFrames` in Pandas. Of note are `pd.concat`, `pd.append`, `pd.merge`, and `pd.join`. 
> **Unless you have a very good reason, use `pd.merge()` to join DataFrames together**

Joins are one of the most ubiquitous operations when working with tabular data. Because of some of the complexities of Pandas data structures, there are many ways of joining dataframes together. By far the most common operation is performing a SQL-like join. To do this operation, use the `pd.merge()` function. 

In [92]:
import random

In [118]:
demographics_df = pd.DataFrame({'patient_id': [1234, 2345, 3456, 4567, 5678, 6789], 
                                'age': [20, 30, 40, 50, 60, 70],
                                'sex': ['M', 'F', 'M', 'F', 'M', 'F']})

vitals_df = pd.DataFrame({'patient_id': [1234, 2345, 3456, 4567, 5678, 6789] * 2,
                          'HR': [random.randint(60, 150) for i in range(12)],
                          'SBP': [random.randint(110, 160) for i in range(12)]})

In [94]:
demographics_df

,patient_id,age,sex
0,1234,20,M
1,2345,30,F
2,3456,40,M
3,4567,50,F
4,5678,60,M
5,6789,70,F


In [95]:
vitals_df

,patient_id,HR,SBP
0,1234,82,148
1,2345,66,157
2,3456,146,144
3,4567,80,138
4,5678,119,147
5,6789,97,152
6,1234,146,131
7,2345,60,140
8,3456,61,127
9,4567,90,113


Let's say we wanted to perform a left join on the `demographics_df` table and get all of the columns from the `vitals_df` table. The correct way to perform the operation is:

In [96]:
pd.merge(demographics_df, vitals_df, how = 'left', left_on = ['patient_id'], right_on = ['patient_id'])

,patient_id,age,sex,HR,SBP
0,1234,20,M,82,148
1,1234,20,M,146,131
2,2345,30,F,66,157
3,2345,30,F,60,140
4,3456,40,M,146,144
5,3456,40,M,61,127
6,4567,50,F,80,138
7,4567,50,F,90,113
8,5678,60,M,119,147
9,5678,60,M,91,125


> The arguments `how = `, `left_on = ` and `right_on = ` must ALWAYS be present in your call. Pandas has a certain way of trying to infer these, and `how = 'inner'` is the default choice, but in order to be explicit, please make sure to include all of them whenever joining DataFrames. 

In [97]:
# Do not use the following !!
demographics_df.merge(vitals_df, how = 'left', left_on = ['patient_id'], right_on = ['patient_id'])

,patient_id,age,sex,HR,SBP
0,1234,20,M,82,148
1,1234,20,M,146,131
2,2345,30,F,66,157
3,2345,30,F,60,140
4,3456,40,M,146,144
5,3456,40,M,61,127
6,4567,50,F,80,138
7,4567,50,F,90,113
8,5678,60,M,119,147
9,5678,60,M,91,125


> **Even though the functionality is equivalent, we prefer to use the `pd.merge` notation rather than calling a method on an existing DataFrame. This goes for any other DataFrame method that can also be expressed as a pandas function call.**

examples include `pd.melt vs DataFrame.melt` and `pd.pivot_table vs DataFrame.pivot`

## What if I want to join on the Index?

Although it is often convenient to join on index, if memory is not an issue, simply 
> **reset the index and join in using the method above**.

This keeps things consistent on our end. However, if there are other limitations, it is ok to use the `left_index` and `right_index` arguments in `pd.merge`. Do **NOT** mix the two if it can be avoided. For example, do not make calls such as:

`pd.merge(left, right, how = 'left', left_on = ['column'], right_index = True)`

This is more headache than it is worth

In [119]:
demographics_df = demographics_df.set_index(['patient_id'])
demographics_df

,age,sex
patient_id,,
1234,20,M
2345,30,F
3456,40,M
4567,50,F
5678,60,M
6789,70,F


In [120]:
vitals_df = vitals_df.set_index(['patient_id'])
vitals_df

,HR,SBP
patient_id,,
1234,65,156
2345,92,145
3456,146,155
4567,138,160
5678,77,138
6789,61,115
1234,61,153
2345,75,159
3456,132,130


In [121]:
using_index = pd.merge(demographics_df, vitals_df, how = 'left', left_index = True, right_index = True)

In [122]:
using_index

,age,sex,HR,SBP
patient_id,,,,
1234,20,M,65,156
1234,20,M,61,153
2345,30,F,92,145
2345,30,F,75,159
3456,40,M,146,155
3456,40,M,132,130
4567,50,F,138,160
4567,50,F,111,155
5678,60,M,77,138


In [123]:
# or just reset the index
demographics_df = demographics_df.reset_index()
vitals_df = vitals_df.reset_index()

In [124]:
pd.merge(demographics_df, vitals_df, how = 'left', left_on = ['patient_id'], right_on = ['patient_id'])

,patient_id,age,sex,HR,SBP
0,1234,20,M,65,156
1,1234,20,M,61,153
2,2345,30,F,92,145
3,2345,30,F,75,159
4,3456,40,M,146,155
5,3456,40,M,132,130
6,4567,50,F,138,160
7,4567,50,F,111,155
8,5678,60,M,77,138
9,5678,60,M,105,144


# Resources 

- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [Pandas Jupyter Notebook Tutorial](https://github.com/dsahduke/Numpy-Pandas)